# Script for crafting Adversarial Examples

In [1]:
import os

import utils.text_processing as tp

In [2]:
# get data from important word detector
%store -r important_words
%store -r sentence_packages

## Methed 2: Mispeeling

#### 1. Create modified Dataset

In [3]:
original_sentences = []
for sentence in sentence_packages:
    original_sentences.append(sentence['original_sentence'])

##### über die gehen
##### word = bleibt gleich
##### typos = Liste alle wörter von index +1 bis zum nächsten Wort, das mit 'Dollar' anfaengt Dieses nicht.

##### dann im nächsten Schritt - für die Länge der Liste Typos über den Satz itereiren und das important word mit dem typo an der index stelle austauschen.





In [4]:
with open(os.path.join('data','birkbeck_corpera_typo.txt'), 'r') as f:
    typods = f.readlines()
    
typodict = {}
for index, item in enumerate(typods):
    if typods[index][0] == '$':
        word = typods[index].rstrip().strip('$')
        typo = typods[index+1].rstrip()
        typodict.update({word : typo})

In [5]:
with open(os.path.join('data','birkbeck_corpera_typo.txt'), 'r') as f:
    typods = f.readlines()
    
typodict = {}
for index, item in enumerate(typods):    
    
    if typods[index][0] == '$':
        word = typods[index].rstrip().strip('$')
        i = 1
        typos = []
        while index+i < len(typods) and typods[index + i][0] is not '$':            
            typos.append(typods[index+i].rstrip())
            i += 1
        typodict.update({word : typos})           

In [6]:
modified_words = []
for word in important_words:
    modified_words.append(tp.to_typo(typodict, word))

In [13]:
modified_sentences = tp.generate_modified_sentences(original_sentences, important_words, modified_words)

In [14]:
len(modified_sentences)

435

#### 2. Import BERT Model

In [8]:
import torch
import random
import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model.eval();

#### 3. Prediction

In [9]:
original_predictions = []
for sentence in original_sentences:
    original_predictions.append(tp.predict_sentiment(model, tokenizer, sentence))

In [16]:
modified_predictions = []

for item in modified_sentences:
    for sentence in item:
        modified_predictions.append(tp.predict_sentiment(model, tokenizer, sentence))

In [20]:
modified_predictions = []

for item in modified_sentences:           
    if isinstance(item, list): 
        modified_prediction_list = []
        for sentence in item:
            modified_prediction_list.append(tp.predict_sentiment(model, tokenizer, sentence))
        modified_predictions.append(modified_prediction_list)               
        continue        
    modified_predictions.append(tp.predict_sentiment(model, tokenizer, sentence))

In [21]:
len(modified_prediction_list)

4

In [22]:
len(modified_predictions)

435

#### 4. Creation of Dataset

In [218]:
ds = pd.DataFrame(list(zip(original_sentences, original_predictions, modified_sentences, modified_predictions)),
                 columns = ['original_sentence', 'original_prediction', 'modified_sentence', 'modified_prediction'])

changed_items = []


mod_pred : [4,
            4,
           [0,4,0],
           4]

für jedes e, Item in enumerate(mod_pred):
    wenn item keine Liste is:
        an changed_items hängen, wenn item != original_score an der stelle e     
        
        sonst (wenn es a Liste is)
            eine neue Liste changed_item_list machen
            und für jeden i, score in der Liste enumerate(item):
                an changed_item_list hängen, wenn score != original_score an der stelle[i]
                              
           und changed item list an die  Liste changed item hängen

dann hab ich eine liste wie mod_pred
    wo nur die drin sind, die sich verändert haben


In [96]:
print(original_predictions[:20])

[4, 4, 3, 0, 4, 2, 4, 4, 3, 3, 4, 4, 4, 4, 3, 4, 4, 3, 2, 1]


In [97]:
print(modified_predictions[:20])

[4, 4, [4, 4, 4], [0, 4, 4, 4], [0, 0, 4, 0, 0, 4], [4, 3, 4], 4, 4, [3, 3, 4, 4], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3], 3, 3, [4], 4, [4, 3, 3, 3, 4, 4, 4, 4, 3, 4, 4, 3, 4], [4, 4, 4], [3, 4, 3, 4, 3, 3, 3, 3, 3, 4, 3, 3, 3], 3, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 1]


In [ ]:
3 --> [4, 4, 4] --> 4er
0 --> [0, 4, 4, 4] --> 4 er
4 --> [0, 0, 4, 0, 0, 4] --> 0er
2 --> [4, 3, 4] --> gleich
3 --> [3, 3, 4, 4] --> 2 4er
3 --> [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3] --> 4
4 --> 3 --> 3
4 --> 3 --> 3
4 --> [4]
4 --> 4

In [104]:
changed_predictions = []
for e, item in enumerate(modified_predictions[:10]):
    if isinstance(item,list):
        changed_predictions_list = []
        for number in item:
            # print(number)
            if number != original_predictions[e]:
                changed_predictions_list.append(number)
            else:
                continue
        changed_predictions.append(changed_predictions_list)
    else:
        print('okok')
        if item != original_predictions[e]:
            changed_predictions.append(item)
            print('okok2')

print(changed_predictions)

okok
okok
okok
okok
[[4, 4, 4], [4, 4, 4], [0, 0, 0, 0], [4, 3, 4], [4, 4], [4]]


In [53]:
changed_modified_predictions = []
for e, item in enumerate(modified_predictions):
    if isinstance(item, list):
        if item != original_predictions[e]:
            changed_modified_predictions.append(item[e])
        
        changed_modified_predictions_list = []
        print(changed_modified_predictions)
        for score in item:
             if score != original_predictions[e]:
                    changed_modified_predictions_list.append(score)
        changed_modified_predictions.append(changed_modified_predictions_list)    

[4]
[4, [4, 4, 4], 4]
[4, [4, 4, 4], 4, [4, 4, 4], 0]


IndexError: list index out of range

In [200]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   original_sentence    435 non-null    object
 1   original_prediction  435 non-null    int64 
 2   modified_sentence    435 non-null    object
 3   modified_prediction  435 non-null    object
dtypes: int64(1), object(3)
memory usage: 13.7+ KB


#### 5. Creation of adversarial Dataset

In [220]:
# gives a list with all modified sentences where prediction is different than original prediction

for index, item in ds.iterrows():
    if isinstance(item.modified_sentence, list):
        newlist = filter(lambda x : item.original_prediction != item.modified_prediction[x[0]], enumerate(item.modified_sentence))
        newlist = list(newlist)
        if len(newlist) != 0:
            _, newlist = zip(*newlist)
            newlist = list(newlist)
            # print(newlist)
        else:
            newlist = None
        ds.at[index,'modified_sentence'] = newlist
        # umschreiben because neewbe lel
        ds.at[index,'modified_prediction'] = [x for x in item.modified_prediction if item.original_prediction != x]
        # print(item)
            
ds = ds[ds.modified_sentence.notnull()]
advds = ds[ds.original_prediction != ds.modified_prediction]

In [221]:
advds[:40]

,original_sentence,original_prediction,modified_sentence,modified_prediction
2,It is in a really good location for riding or ...,3,[It is in a really good location for riding or...,"[4, 4, 4]"
3,I cannot recommend this place enough!.,0,"[I cannto recommend this place enough!., I can...","[4, 4, 4]"
4,3 friends and I visited 't Materke in April 20...,4,[3 friends and I visited 't Materke i April 20...,"[0, 0, 0, 0]"
5,2 of us had stayed before so knew to expect a ...,2,[2 of us had stayed before so knew to espect a...,"[4, 3, 4]"
8,Edith and Mario are lovely and couldn't have b...,3,[Edith and Mario are lovely and coundnt have b...,"[4, 4]"
9,"The rooms are comfortable, clean and have plen...",3,"[The rooms are comfortable, clean and hove ple...",[4]
10,Breakfast is incredible and a highlight of any...,4,Breakfast is incredible and a highlight of any...,3
11,Ideal to fuel a day's bike riding.,4,Ideal to fuel a day's bike riding.,3
14,"- Very nice place to stay, one of the best in ...",3,"[- Very nice place to stay, one of the best in...","[4, 4, 4, 4, 4, 4, 4, 4]"
16,New bathroom with a very good shower.,4,"[New bathroom with a an good shower., New bath...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]"


In [208]:
advds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 2 to 434
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   original_sentence    330 non-null    object
 1   original_prediction  330 non-null    int64 
 2   modified_sentence    330 non-null    object
 3   modified_prediction  330 non-null    object
dtypes: int64(1), object(3)
memory usage: 12.9+ KB


In [222]:
 advds.to_json(r'data/adversarial_dataset_multitypos.json', orient='records')